# Python OOP Fundamentals

This notebook teaches Object-Oriented Programming (OOP) basics in Python by **building reusable classes** step-by-step.

By the end, we'll have a working mini card game using these classes:
- `Card`
- `Deck`
- `Hand`
- `Player`
- `Game`

Along the way we’ll learn:
- classes and objects
- `__init__` and attributes
- methods
- encapsulation (keeping logic inside the right class)
- composition (objects containing other objects)
- simple inheritance

## Why OOP?

OOP helps us organize code by bundling:
- **data** (attributes / state)
- **behavior** (methods)

Instead of scattered variables and functions, we model real concepts:
- A `Card` has a `rank` and `suit`
- A `Deck` *contains* many `Card`s and can `shuffle()` and `draw()`
- A `Player` has a `Hand` and can `play_card()`

## Learning Objectives

By the end of this notebook, you should be able to:
- Explain what a class and object are
- Define attributes and methods
- Use `__init__` to initialize objects
- Describe relationships between classes (composition)
- Trace how multiple classes work together in a program


In [ ]:
# This is your class
class Greeter:
    # This is a method
    def say_hello(self):
        print("Hello!")
    
    def say_goodbye(self):
        print("Goodbye!")

# This is an object
g = Greeter()
g.say_hello()
g.say_goodbye()

g2 = Greeter()
g2.say_hello()

In [ ]:
class Car:
    def drive(self):
        print("Vroom Vroom")

tesla = Car()
mercedes = Car()
BYD = Car()
perfected_unicorn = Car()

tesla.drive()

## `__init__`, `self`, and attributes

- `__init__` runs when you create a new object.
- `self` refers to "this specific object."
- Attributes belong to the object (each object gets its own copy).


In [ ]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank # attribute
        self.suit = suit # attribute

    def describe(self):
        return f"{self.rank} of {self.suit}"
    
c1 = Card("A", "Spades")
c2 = Card("10", "Hearts")

c1.describe()

The following is incorrect, since there is no 'self' in front of the attributes:

In [ ]:
class Dog:
    def __init__(self, breed, age):
        breed = breed
        age = age

## Encapsulation

Instead of writing logic *outside* the object…

Bad style:
```python
# scattered logic
if card.rank == "A": ...
```

Better OOP style: put logic inside the class and make the class responsible for its own behavior

Next: Teach `Card` how to compute its own value.

In [ ]:
class Card:
    RANK_VALUES = {
        "2": 2, "3": 3, "4": 4, "5": 5, "6": 6,
        "7": 7, "8": 8, "9": 9, "10": 10,
        "J": 11, "Q": 12, "K": 13, "A": 14
    }

    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def describe(self):
        return f"{self.rank} of {self.suit}"
    
    def value(self):
        return Card.RANK_VALUES[self.rank]
    
c = Card("K", "Diamonds")
print(c.describe(), "| value =", c.value())

In [ ]:
c1 = Card("K", "Diamonds")
c2 = Card("A", "Spades")

c2.value() > c1.value()

## Composition (building bigger systems from smaller classes)

Now we’ll make a `Deck` that contains many `Card` objects.

- a `Deck` is not just data; it has behavior (`shuffle`, `draw`)
- it manages its internal cards

In [ ]:
import random

class Deck:
    SUITS = ["Hearts", "Diamonds", "Clubs", "Spades"]
    RANKS = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]

    def __init__(self):
        self._cards = [] # underscore means: "internal use"
        self.reset()

    def reset(self):
        self._cards = [Card(rank, suit) for suit in Deck.SUITS for rank in Deck.RANKS]

    def shuffle(self):
        random.shuffle(self._cards)

    def draw(self):
        if not self._cards:
            return None
        return self._cards.pop()
    
    def cards_left(self):
        return len(self._cards)
    
deck = Deck()
deck.shuffle()

print(deck.draw().describe())
print("cards left:", deck.cards_left())

## Hand

A `Hand` represents the cards a player holds.
It should handle things like:
- adding cards
- playing a card
- showing contents
- computing strongest card, etc.

Notice how we reuse `Card` again.


In [ ]:
class Hand:
    def __init__(self):
        self._cards = []

    def add(self, card):
        if card is not None:
            self._cards.append(card)

    def size(self):
        return len(self._cards)
    
    def show(self):
        """Return pretty strings for display."""
        return [c.describe() for c in self._cards]
    
    def get_cards(self):
        """Expose cards as a read-only copy for decision making"""
        return list(self._cards)
    
    def play_index(self, idx):
        """Remove and return the card at position idx"""
        if idx < 0 or idx >= len(self._cards):
            return None
        return self._cards.pop(idx)

h = Hand()
h.add(Card("A", "Spades"))
h.add(Card("5", "Hearts"))
print("Hand:", h.show())
print(h.get_cards())
print("Played:", h.play_index(0).describe())
print("Hand now:", h.show())

## Player

A `Player` object should store:
- name
- a `Hand`
- score (for games that score points)

This shows a common OOP structure:
- `Player` *contains* a `Hand`


In [ ]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()
        self.score = 0

    def draw_from(self, deck, n=1):
        for _ in range(n):
            self.hand.add(deck.draw())
    
    def play_card(self, idx):
        return self.hand.play_index(idx)
    
    def add_point(self):
        self.score += 1

p = Player("Yousen")
deck = Deck()
deck.shuffle()
p.draw_from(deck, n=3)
print(p.name, "has:", p.hand.show())

In [ ]:
class Game:
    def __init__(self, player1_name, player2_name):
        self.deck = Deck()
        self.deck.shuffle()
        self.players = [
            Player(player1_name),
            Player(player2_name)
        ]

        # Deal 5 cards each
        for player in self.players:
            player.draw_from(self.deck, n=5)

    def play_round(self):
        p1, p2 = self.players

        card1 = p1.play_card(0)
        card2 = p2.play_card(0)

        print(f"{p1.name} plays {card1.describe()}")
        print(f"{p2.name} plays {card2.describe()}")

        if card1.value() > card2.value():
            print(f"{p1.name} wins the round!\n")
            p1.add_point()
        elif card2.value() > card1.value():
            print(f"{p2.name} wins the round!\n")
        else:
            print("It's a tie!")

    def play_game(self):
        rounds = self.players[0].hand.size()

        for _ in range(rounds):
            self.play_round()

        self.show_winner()

    